<a href="https://colab.research.google.com/github/sahupra1357/LLMAgents/blob/main/Groq_StructuredOutput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

In [3]:
%pip install -U groq instructor python-dotenv -q

In [5]:
import instructor
from dotenv import load_dotenv
from pydantic import BaseModel
from groq import Groq

# Load the Groq API key from .env file
load_dotenv()

# Describe the desired output schema using pydantic models
class UserInfo(BaseModel):
    name: str
    age: int
    email: str

# The text to extract data from
text = """
John Doe, a 35-year-old software engineer from New York, has been working with large language models for several years.
His email address is johndoe@example.com.
"""

# Patch Groq() with instructor, this is where the magic happens!
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

# Call the API
user_info = client.chat.completions.create(
    #model="llama-3.1-70b-versatile",
    model="deepseek-r1-distill-llama-70b",
    response_model=UserInfo, # Specify the response model
    messages=[
        {"role": "system", "content": "Your job is to extract user information from the given text."},
        {"role": "user", "content": text}
    ],
    temperature=0.65,
)

print(f"Name: {user_info.name}")
print(f"Age: {user_info.age}")
print(f"Email: {user_info.email}")

Name: John Doe
Age: 35
Email: johndoe@example.com


In [6]:
from pprint import pprint

import instructor
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from groq import Groq

# Load the Groq API key from .env file
load_dotenv()

prompt = """
I am designing a weather agent. This agent can talk to the user and also fetch latest weather information.
It has access to the `get_weather_info` tool with the following JSON schema:
{json_schema}

I want you to write some examples for `get_weather_info` and see if this functionality works correctly and can handle all the cases.
Now given the information so far and the JSON schema of the provided tool, write {num} examples.
Make sure each example is varied enough to cover common ways of requesting for this functionality.
Make sure you fill the function parameters with the correct types when generating the output examples.
Make sure your output is valid JSON.
"""

In [7]:
class Example(BaseModel):
    input_text: str = Field(description="The example text")
    tool_name: str = Field(description="The tool name to call for this example")
    tool_parameters: str = Field(description="An object containing the key-value pairs for the parameters of this tool as a JSON serializbale STRING, make sure it is valid JSON and parameter values are of the correct type according to the tool schema")

class ResponseModel(BaseModel):
    examples: list[Example]

In [8]:
# The schema for get_weather_info tool
tool_schema = {
    "name": "get_weather_info",
    "description": "Get the weather information for any location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location for which we want to get the weather information (e.g. New York)"
            }
        },
        "required": ["location"]
    }
}

# Patch Groq() with instructor, this is where the magic happens!
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

# Call the API with our custom prompt and ResponseModel
response = client.chat.completions.create(
    #model="llama-3.1-70b-versatile",
    model="deepseek-r1-distill-llama-70b",
    response_model=ResponseModel, # Specify the response model
    messages=[
        {
            "role": "system",
            "content": prompt.format(json_schema=tool_schema, num=5), # Pass the tool schema and number of examples to the prompt
        },
    ],
    temperature=0.65,
    max_tokens=8000,
)

print(type(response))
pprint(response.examples)

<class '__main__.ResponseModel'>
[Example(input_text="What's the weather like in New York?", tool_name='get_weather_info', tool_parameters='{"location": "New York"}'),
 Example(input_text='Can you tell me the weather in London today?', tool_name='get_weather_info', tool_parameters='{"location": "London"}'),
 Example(input_text="What's the weather in Tokyo on Friday?", tool_name='get_weather_info', tool_parameters='{"location": "Tokyo"}'),
 Example(input_text='Is it raining in Paris this weekend?', tool_name='get_weather_info', tool_parameters='{"location": "Paris"}'),
 Example(input_text="What's the weather like in Chicago?", tool_name='get_weather_info', tool_parameters='{"location": "Chicago"}')]
